In [ ]:
pip install pydicom nibabel numpy torch torchvision segmentation-models-pytorch scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install albumentations

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 9
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_unetplusplus_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'unetplusplus_model_knee_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)

    print("Model saved successfully.")


print("Training U-Net++ ..")
train_unetplusplus_model(encoder_name="resnet18")


Training U-Net++ ..


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [1/100] - Train IoU: 0.0615 - Validation IoU: 0.0853


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [2/100] - Train IoU: 0.1074 - Validation IoU: 0.1087


...

Epoch [98/100] - Train IoU: 0.9901 - Validation IoU: 0.9414


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [99/100] - Train IoU: 0.9900 - Validation IoU: 0.9395


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [100/100] - Train IoU: 0.9901 - Validation IoU: 0.9402
Model saved successfully.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 9
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_linknet_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.Linknet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'linknet_model_knee_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)


    print("Model saved successfully.")


print("Training LinkNet ..")
train_linknet_model(encoder_name="resnet18")


Training LinkNet ..


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [1/100] - Train IoU: 0.1305 - Validation IoU: 0.0288


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [2/100] - Train IoU: 0.1534 - Validation IoU: 0.0288


...

Epoch [98/100] - Train IoU: 0.7787 - Validation IoU: 0.7406


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [99/100] - Train IoU: 0.7840 - Validation IoU: 0.7439


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [100/100] - Train IoU: 0.7848 - Validation IoU: 0.7452
Model saved successfully.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 9
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))  # Resize to match model input size

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_pspnet_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.PSPNet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'pspnet_model_knee_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)

    print("Model saved successfully.")


print("Training PSPNet ..")
train_pspnet_model(encoder_name="resnet18")


Training PSPNet ..


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [1/100] - Train IoU: 0.0905 - Validation IoU: 0.1097


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [2/100] - Train IoU: 0.4125 - Validation IoU: 0.5950


...

Epoch [98/100] - Train IoU: 0.9619 - Validation IoU: 0.9313


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [99/100] - Train IoU: 0.9633 - Validation IoU: 0.9313


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [100/100] - Train IoU: 0.9651 - Validation IoU: 0.9301
Model saved successfully.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 9
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_fpn_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.FPN(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'fpn_model_knee_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)

    print("Model saved successfully.")


print("Training FPN ..")
train_fpn_model(encoder_name="resnet18")


Training FPN ..


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [1/100] - Train IoU: 0.3603 - Validation IoU: 0.6039


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [2/100] - Train IoU: 0.6481 - Validation IoU: 0.6040


...

Epoch [98/100] - Train IoU: 0.9707 - Validation IoU: 0.9090


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [99/100] - Train IoU: 0.9694 - Validation IoU: 0.9124


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [100/100] - Train IoU: 0.9708 - Validation IoU: 0.9167
Model saved successfully.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 9
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_pan_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.PAN(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'pan_model_knee_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)


    print("Model saved successfully.")


print("Training PAN ..")
train_pan_model(encoder_name="resnet18")


Training PAN ..


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [1/100] - Train IoU: 0.0249 - Validation IoU: 0.0225


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [2/100] - Train IoU: 0.0837 - Validation IoU: 0.0245


...

Epoch [98/100] - Train IoU: 0.9743 - Validation IoU: 0.9210


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [99/100] - Train IoU: 0.9733 - Validation IoU: 0.9219


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [100/100] - Train IoU: 0.9756 - Validation IoU: 0.9220
Model saved successfully.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 9
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_manet_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.MAnet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'manet_model_knee_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)


    print("Model saved successfully.")


print("Training MAnet ..")
train_manet_model(encoder_name="resnet18")


Training MAnet ..


Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [1/100] - Train IoU: 0.0241 - Validation IoU: 0.0322


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [2/100] - Train IoU: 0.0337 - Validation IoU: 0.0409


...

Epoch [98/100] - Train IoU: 0.9841 - Validation IoU: 0.9287


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [99/100] - Train IoU: 0.9833 - Validation IoU: 0.9273


/home/ealam/.local/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


Epoch [100/100] - Train IoU: 0.9852 - Validation IoU: 0.9279
Model saved successfully.
